In [4]:
!pip install opencv-python numpy pytesseract

In [5]:
import cv2
import numpy as np
import pytesseract
from pathlib import Path
import os

# Chuyển working directory về thư mục chứa notebook
os.chdir(r'h:\20262027Period\Programming\Quarter2\Python\5')

# Khai báo đường dẫn Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Tạo thư mục output
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)

# Đọc ảnh
img = cv2.imread('Picture1.png')
if img is None:
    raise FileNotFoundError("Không tìm thấy file 'Picture1.png' trong thư mục hiện tại!")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Bước 1: Loại bỏ chữ để tìm các góc vuông
blur = cv2.GaussianBlur(gray, (5, 5), 0)
_, thresh = cv2.threshold(blur, 200, 255, cv2.THRESH_BINARY)

# Bước 2: Tạo template cho góc trên-trái và dưới-phải
template_size = 20

# Template góc trên-trái (L ngược)
template_tl = np.ones((template_size, template_size), dtype=np.uint8) * 255
template_tl[0:3, :] = 0
template_tl[:, 0:3] = 0

# Template góc dưới-phải (L)
template_br = np.ones((template_size, template_size), dtype=np.uint8) * 255
template_br[-3:, :] = 0
template_br[:, -3:] = 0

# Tìm các góc
res_tl = cv2.matchTemplate(thresh, template_tl, cv2.TM_CCOEFF_NORMED)
res_br = cv2.matchTemplate(thresh, template_br, cv2.TM_CCOEFF_NORMED)

threshold = 0.6
loc_tl = np.where(res_tl >= threshold)
loc_br = np.where(res_br >= threshold)

corners_tl = list(zip(*loc_tl[::-1]))  # (x, y)
corners_br = list(zip(*loc_br[::-1]))

# Hàm lọc các điểm trùng lặp gần nhau (Non-Maximum Suppression)
def nms_corners(corners, min_dist=30):
    if not corners:
        return []
    filtered = [corners[0]]
    for c in corners[1:]:
        too_close = False
        for f in filtered:
            if abs(c[0] - f[0]) < min_dist and abs(c[1] - f[1]) < min_dist:
                too_close = True
                break
        if not too_close:
            filtered.append(c)
    return filtered

corners_tl = nms_corners(corners_tl)
corners_br = nms_corners(corners_br)

print(f"Tìm thấy {len(corners_tl)} góc trên-trái, {len(corners_br)} góc dưới-phải")

# Sắp xếp theo y trước, rồi x (từ trên xuống dưới, trái sang phải)
corners_tl.sort(key=lambda c: (c[1], c[0]))
corners_br.sort(key=lambda c: (c[1], c[0]))

# Bước 3: Ghép cặp góc TL-BR gần nhất
def pair_corners(tl_list, br_list, template_size):
    pairs = []
    used_br = set()
    for tl in tl_list:
        best_br = None
        best_dist = float('inf')
        for j, br in enumerate(br_list):
            if j in used_br:
                continue
            if br[0] > tl[0] and br[1] > tl[1]:
                dist = (br[0] - tl[0])**2 + (br[1] - tl[1])**2
                if dist < best_dist:
                    best_dist = dist
                    best_br = j
        if best_br is not None:
            used_br.add(best_br)
            pairs.append((tl, br_list[best_br]))
    return pairs

pairs = pair_corners(corners_tl, corners_br, template_size)
print(f"Ghép được {len(pairs)} cặp góc")

# Crop và xử lý từng vùng
regions = []
for tl, br in pairs:
    x1, y1 = tl
    x2, y2 = br[0] + template_size, br[1] + template_size

    if x2 > x1 and y2 > y1 and x2 <= img.shape[1] and y2 <= img.shape[0]:
        region = img[y1:y2, x1:x2]
        if region.size > 0:
            regions.append((region, (x1, y1, x2, y2)))

print(f"Crop được {len(regions)} vùng")

# Phân loại và lưu
saved_count = 0
for i in range(0, len(regions), 2):
    if i + 1 < len(regions):
        # Vùng lẻ: Họ tên
        name_region = regions[i][0]
        if name_region.size == 0:
            continue
        name_gray = cv2.cvtColor(name_region, cv2.COLOR_BGR2GRAY)
        name_text = pytesseract.image_to_string(name_gray, lang='vie').strip()
        # Loại bỏ ký tự đặc biệt không hợp lệ trong tên file
        name_text = ''.join(c for c in name_text if c not in r'\/:*?"<>|')

        # Vùng chẵn: Chữ ký
        signature_region = regions[i + 1][0]

        # Lưu file chữ ký với tên người
        if name_text:
            filename = os.path.join(output_dir, f"{name_text}.png")
            cv2.imwrite(filename, signature_region)
            print(f"Đã lưu: {filename}")
            saved_count += 1
        else:
            print(f"Vùng {i}: không đọc được tên")

print(f"\nHoàn thành! Đã lưu {saved_count} file chữ ký vào thư mục '{output_dir}'")

Tìm thấy 110 góc trên-trái, 120 góc dưới-phải
Ghép được 86 cặp góc
Crop được 86 vùng
Vùng 0: không đọc được tên
Vùng 2: không đọc được tên
Vùng 4: không đọc được tên
Vùng 6: không đọc được tên
Vùng 8: không đọc được tên
Vùng 10: không đọc được tên
Vùng 12: không đọc được tên
Vùng 14: không đọc được tên
Vùng 16: không đọc được tên
Vùng 18: không đọc được tên
Vùng 20: không đọc được tên
Vùng 22: không đọc được tên
Vùng 24: không đọc được tên
Vùng 26: không đọc được tên
Đã lưu: output\Văn An.png
Vùng 30: không đọc được tên
Vùng 32: không đọc được tên
Vùng 34: không đọc được tên
Vùng 36: không đọc được tên
Vùng 38: không đọc được tên
Vùng 40: không đọc được tên
Vùng 42: không đọc được tên
Vùng 44: không đọc được tên
Vùng 46: không đọc được tên
Vùng 48: không đọc được tên
Đã lưu: output\II.png
Vùng 52: không đọc được tên
Vùng 54: không đọc được tên
Vùng 56: không đọc được tên
Vùng 58: không đọc được tên
Vùng 60: không đọc được tên
Vùng 62: không đọc được tên
Đã lưu: output\. lœ,.png
Vùng 66

# Hướng dẫn

### 1. Loại bỏ chữ khỏi file signlist
* Sử dụng `colorcvt`, `grayscale`, `blur` và `threshold`.

### 2. Tìm các góc vuông
* Sử dụng hàm `matchTemplate` của CV2.
* Chỉ cần tìm các góc **trên-trái** và **dưới-phải**.

### 3. Crop và xử lý vùng dữ liệu
* Crop những vùng giới hạn bởi từng cặp (trên-trái, dưới-phải).
* Phân loại vùng:
    * **Vùng Họ tên (lẻ):** Dùng Tesseract để lấy văn bản.
    * **Vùng Chữ ký (chẵn):** Lưu vùng chữ ký với tên là văn bản tương ứng từ vùng Họ tên.
